<a href="https://colab.research.google.com/github/jcvasquezbetancur/DS-colab/blob/master/sqlalchemy_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sqlalchemy -q

In [ ]:
import sqlalchemy as db

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column,Sequence, Integer, String
engine = create_engine('sqlite:///college.db', echo = True)
meta = MetaData()

students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String),
)
meta.create_all(engine)
print("these are columns in our table %s" %(students.columns.keys()))


2021-06-16 18:20:10,591 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-16 18:20:10,593 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2021-06-16 18:20:10,598 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-16 18:20:10,602 INFO sqlalchemy.engine.Engine COMMIT
these are columns in our table ['id', 'name', 'lastname']


#insertion

In [ ]:
ins = students.insert()
ins = students.insert().values(name = 'Juan', lastname = 'Vasquez')
conn = engine.connect()
result = conn.execute(ins)

2021-06-16 17:45:43,501 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-16 17:45:43,506 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ('Juan', 'Vasquez')
2021-06-16 17:45:43,509 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
conn.execute(students.insert(), [
   {'name':'Bhaskar', 'lastname' : 'guptha'},
   {'name':'vibhav','lastname' : 'kumar'},
   {'name':'prudhvi','lastname' : 'varma'},
   {'name':'manoj','lastname' : 'varma'},
])

2021-06-16 17:47:05,850 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-16 17:47:05,852 INFO sqlalchemy.engine.Engine [generated in 0.00231s] (('Bhaskar', 'guptha'), ('vibhav', 'kumar'), ('prudhvi', 'varma'), ('manoj', 'varma'))
2021-06-16 17:47:05,856 INFO sqlalchemy.engine.Engine COMMIT


#delete a matching


In [ ]:
conn = engine.connect()
stmt = students.delete().where(students.c.name == 'manoj')
conn.execute(stmt)
s = students.select()
conn.execute(s).fetchall()

2021-06-16 18:20:20,336 INFO sqlalchemy.engine.Engine DELETE FROM students WHERE students.name = ?
2021-06-16 18:20:20,338 INFO sqlalchemy.engine.Engine [generated in 0.00207s] ('manoj',)
2021-06-16 18:20:20,345 INFO sqlalchemy.engine.Engine COMMIT
2021-06-16 18:20:20,351 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-16 18:20:20,353 INFO sqlalchemy.engine.Engine [generated in 0.00373s] ()


[(1, 'Dr.Juan C.', 'Vasquez'),
 (2, 'Bhaskar', 'guptha'),
 (3, 'vibhav', 'kumar'),
 (4, 'prudhvi', 'varma')]

#update an entrie

In [ ]:
stmt=students.update().where(students.c.name=='Juan').values(name='Dr.Juan C.')
conn.execute(stmt)
s = students.select()
conn.execute(s).fetchall()

2021-06-16 17:49:46,179 INFO sqlalchemy.engine.Engine UPDATE students SET name=? WHERE students.name = ?
2021-06-16 17:49:46,187 INFO sqlalchemy.engine.Engine [generated in 0.00841s] ('Dr.Juan C.', 'Juan')
2021-06-16 17:49:46,191 INFO sqlalchemy.engine.Engine COMMIT
2021-06-16 17:49:46,207 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-16 17:49:46,208 INFO sqlalchemy.engine.Engine [cached since 68.91s ago] ()


[(1, 'Dr.Juan C.', 'Vasquez'),
 (2, 'Bhaskar', 'guptha'),
 (3, 'vibhav', 'kumar'),
 (4, 'prudhvi', 'varma')]

# filtering/retrieval

In [ ]:
choosen_lastname =[ 'kumar', 'varma']
s = students.select().where(students.c.lastname.in_(choosen_lastname)  )
result = conn.execute(s)
for row in result:
   print (row)

2021-06-16 17:59:43,237 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.lastname IN (?, ?)
2021-06-16 17:59:43,240 INFO sqlalchemy.engine.Engine [generated in 0.00283s] ('kumar', 'varma')
(3, 'vibhav', 'kumar')
(4, 'prudhvi', 'varma')


# Let's bind a session to connect directly.

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [ ]:
session=Session()

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [ ]:
class studentclass(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    lastname = Column(String(50))
   
    def __repr__(self):
        return "<User(name='%s', lastname='%s')>" % (
                                self.name, self.lastname)

In [ ]:
new_user = studentclass(name='ed', lastname='edslastname')
session.add(new_user)

In [ ]:
first_student = session.query(students).filter_by(lastname='Vasquez').first()
first_student

2021-06-16 18:26:07,809 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.lastname AS students_lastname 
FROM students 
WHERE students.lastname = ?
 LIMIT ? OFFSET ?
2021-06-16 18:26:07,817 INFO sqlalchemy.engine.Engine [generated in 0.00847s] ('Vasquez', 1, 0)


(1, 'Dr.Juan C.', 'Vasquez')

In [ ]:
session.add_all([
...     studentclass(name='wendy', lastname='Williams')])
session.commit()

2021-06-16 18:28:18,560 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-16 18:28:18,567 INFO sqlalchemy.engine.Engine [cached since 166.2s ago] ('wendy', 'Williams')
2021-06-16 18:28:18,571 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
new_user.id

5

In [ ]:
session.execute(students.select()).fetchall()

2021-06-16 18:34:17,443 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-16 18:34:17,446 INFO sqlalchemy.engine.Engine [generated in 0.00386s] ()


[(1, 'Dr.Juan C.', 'Vasquez'),
 (2, 'Bhaskar', 'guptha'),
 (3, 'vibhav', 'kumar'),
 (4, 'prudhvi', 'varma'),
 (5, 'ed', 'edslastname'),
 (6, 'wendy', 'Williams')]

In [ ]:
for instance in session.query(students).order_by(studentclass.name):
  print(instance.name, instance.lastname)


2021-06-16 18:38:49,532 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.lastname AS students_lastname 
FROM students ORDER BY students.name
2021-06-16 18:38:49,535 INFO sqlalchemy.engine.Engine [generated in 0.00404s] ()
Bhaskar guptha
Dr.Juan C. Vasquez
ed edslastname
prudhvi varma
vibhav kumar
wendy Williams


In [ ]:
for instance in session.query(students).order_by(studentclass.name.ilike('')):
  print(instance.name, instance.lastname)

2021-06-16 18:42:09,781 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.lastname AS students_lastname 
FROM students ORDER BY lower(students.name) LIKE lower(?)
2021-06-16 18:42:09,785 INFO sqlalchemy.engine.Engine [cached since 26.09s ago] ('',)
Dr.Juan C. Vasquez
Bhaskar guptha
vibhav kumar
prudhvi varma
ed edslastname
wendy Williams
